In [13]:

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import os
from fake_useragent import UserAgent
from random import randint
import pandas as pd
import numpy as np
import requests
import pandas as pd
import re
import convert_numbers

In [ ]:
df = pd.read_excel('spring_product_update2.xlsx')
df['price1'] = [convert_numbers.hindi_to_english(x) for x in df['price']]
df['price1']

In [16]:
df.loc[df['special_price'].isnull(), 'special_price'] = 0

,Unnamed: 0,Unnamed: 0.1,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,...,name,price,special_price,link_url,description,base_image,add_images,cat1,cat2,price1
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,طاولة خشب شكل دراجة حديد,٦٩٠,NaN,https://spring01.com/طاولة-خشب-شكل-دراجة-حديد/...,.الارتفاع. 94..سنتم العرض..150..سنتممساحة السط...,https://cdn.salla.sa/YzvAn/cEWho575pdCJSsIBH5i...,https://cdn.salla.sa/YzvAn/pgfbcjjKXHHPy4J739t...,الحديقة والزرع,NaN,690
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,طاولة حديد سطح خشب,١٨٠,NaN,https://spring01.com/طاولة-حديد-سطح-خشب/p20649...,.الارتفاع44..سنتم مقاس السطح .. 32 ..سنتم,https://cdn.salla.sa/YzvAn/oOCWlNR1cV6bhtPqKyp...,https://cdn.salla.sa/YzvAn/ZgRHL8TmTrHEg1PhjGE...,طاولات وكراسي,طاولات,180
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,طقم طاولات حديد سطح خشب رخامي 2 قطعة,٥٣٠,NaN,https://spring01.com/طقم-طاولات-حديد-سطح-خشب-ر...,.ارتفاع كبير. 43..نتممقاس السطح ... 90 * 47..س...,https://cdn.salla.sa/YzvAn/RBfYWowbP481pMKZLfZ...,https://cdn.salla.sa/YzvAn/X6rXAsIdQwFlodd45jq...,طاولات وكراسي,طاولات,530
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,طقم طاولات حديد سطح خشب رخامي 2 قطعة,٥٣٠,NaN,https://spring01.com/طقم-طاولات-حديد-سطح-خشب-ر...,ارتفاع كبير. 43..نتممقاس السطح ... 90 * 47..سن...,https://cdn.salla.sa/YzvAn/qvprqAzCHO9IOAwGH2y...,https://cdn.salla.sa/YzvAn/iWvLiPSOYcHMb1xyF7a...,طاولات وكراسي,طاولات,530
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,طقم طاولات حديد سطح خشب 2 قطعة,٥٣٠,NaN,https://spring01.com/طقم-طاولات-حديد-سطح-خشب-2...,..ارتفاع كبير. 43..نتممقاس السطح ... 120 * 48....,https://cdn.salla.sa/YzvAn/8qJwZAUTrLQ3EAzWIUr...,https://cdn.salla.sa/YzvAn/GK6HE4j5iVSMFHnv0fE...,طاولات وكراسي,طاولات,530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,منفاخ كفرات سيارة 2 بستم 12,١١٥,NaN,https://spring01.com/منفاخ-كفرات-سيارة-2-بستم-...,NaN,https://cdn.salla.sa/YzvAn/HOYRn7bq7rpZyOLBxlX...,NaN,منتجات متنوعة,NaN,115
581,581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,عدة زراعية لون اسود قدوم,١٢,NaN,https://spring01.com/عدة-زراعية-لون-اسود-قدوم/...,NaN,https://cdn.salla.sa/YzvAn/XluUMlNEVPjtcuF1hIq...,NaN,منتجات متنوعة,NaN,12
584,584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,عطر,٤٨,NaN,https://spring01.com/عطر/p460862789,NaN,https://cdn.salla.sa/YzvAn/CRukC1TZyb2JD0ncxkp...,NaN,منتجات متنوعة,NaN,48
585,585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,عطر,٤٨,NaN,https://spring01.com/عطر/p900985799,NaN,https://cdn.salla.sa/YzvAn/ve1zc9FV4xBUXR9FBb3...,NaN,منتجات متنوعة,NaN,48


In [2]:
urls = [
    {'cat1': 'طاولات وكراسي', 'cat2': 'طاولات', 'cat3': '', 'url': 'https://spring01.com/%D8%B7%D8%A7%D9%88%D9%84%D8%A7%D8%AA/c2022184649'},
    {'cat1': 'طاولات وكراسي', 'cat2': 'كراسي', 'cat3': '', 'url': 'https://spring01.com/%D9%83%D8%B1%D8%A7%D8%B3%D9%8A/c607278283'},
    {'cat1': 'طاولات وكراسي', 'cat2': 'طقم جلسة', 'cat3': '', 'url': 'https://spring01.com/%D8%B7%D9%82%D9%85-%D8%AC%D9%84%D8%B3%D8%A9/c1139118805'},
    
    {'cat1': 'دواليب', 'cat2': 'دواليب مدخل', 'cat3': '', 'url': 'https://spring01.com/%D8%AF%D9%88%D8%A7%D9%84%D9%8A%D8%A8-%D9%85%D8%AF%D8%AE%D9%84/c1873793239'},
    {'cat1': 'دواليب', 'cat2': 'أدراج وصناديق', 'cat3': '', 'url': 'https://spring01.com/%D8%A3%D8%AF%D8%B1%D8%A7%D8%AC-%D9%88%D8%B5%D9%86%D8%A7%D8%AF%D9%8A%D9%82/c1698577874'},
    {'cat1': 'دواليب', 'cat2': 'دواليب جزم', 'cat3': '', 'url': 'https://spring01.com/%D8%AF%D9%88%D8%A7%D9%84%D9%8A%D8%A8-%D8%AC%D8%B2%D9%85/c14711772'},
    
    {'cat1': 'رفوف-ستنادات-فازات', 'cat2': 'رفوف وستاندات', 'cat3': '', 'url': 'https://spring01.com/%D8%B1%D9%81%D9%88%D9%81-%D9%88%D8%B3%D8%AA%D8%A7%D9%86%D8%AF%D8%A7%D8%AA/c1389271773'},
    {'cat1': 'رفوف-ستنادات-فازات', 'cat2': 'فازات', 'cat3': '', 'url': 'https://spring01.com/%D9%81%D8%A7%D8%B2%D8%A7%D8%AA/c334912482'},
    
    {'cat1': 'مفارش-مخدات', 'cat2': 'مفارش أرضية', 'cat3': '', 'url': 'https://spring01.com/%D9%85%D9%81%D8%A7%D8%B1%D8%B4-%D8%A3%D8%B1%D8%B6%D9%8A%D8%A9/c363359312'},
    {'cat1': 'مفارش-مخدات', 'cat2': 'مخدات', 'cat3': '', 'url': 'https://spring01.com/%D9%85%D8%AE%D8%AF%D8%A7%D8%AA/c1735822161'},
    
    {'cat1': 'لوحات وديكورات جدارية', 'cat2': 'لوحات', 'cat3': '', 'url': 'https://spring01.com/%D9%84%D9%88%D8%AD%D8%A7%D8%AA/c331965220'},
    {'cat1': 'لوحات وديكورات جدارية', 'cat2': 'مفارش وستائر جدارية', 'cat3': '', 'url': 'https://spring01.com/%D9%85%D9%81%D8%A7%D8%B1%D8%B4-%D9%88%D8%B3%D8%AA%D8%A7%D8%A6%D8%B1-%D8%AC%D8%AF%D8%A7%D8%B1%D9%8A%D8%A9/c1529278240'},
    {'cat1': 'لوحات وديكورات جدارية', 'cat2': 'ديكورات جدارية', 'cat3': '', 'url': 'https://spring01.com/%D8%AF%D9%8A%D9%83%D9%88%D8%B1%D8%A7%D8%AA-%D8%AC%D8%AF%D8%A7%D8%B1%D9%8A%D8%A9/c889392673'},
    
    {'cat1': 'الساعات والمرايا', 'cat2': 'ساعات', 'cat3': '', 'url': 'https://spring01.com/%D8%B3%D8%A7%D8%B9%D8%A7%D8%AA/c580021036'},
    {'cat1': 'الساعات والمرايا', 'cat2': 'مرايا', 'cat3': '', 'url': 'https://spring01.com/%D9%85%D8%B1%D8%A7%D9%8A%D8%A7/c2088274477'},
    
    {'cat1': 'السلات و أواني التقديم', 'cat2': 'السلات', 'cat3': '', 'url': 'https://spring01.com/%D8%A7%D9%84%D8%B3%D9%84%D8%A7%D8%AA/c404932655'},
    {'cat1': 'السلات و أواني التقديم', 'cat2': 'صحون تقديم', 'cat3': '', 'url': 'https://spring01.com/%D8%B5%D8%AD%D9%88%D9%86-%D8%AA%D9%82%D8%AF%D9%8A%D9%85/c1205138985'},
    
    {'cat1': 'ركن القهوة', 'cat2': 'دواليب وعربات القهوة', 'cat3': '', 'url': 'https://spring01.com/%D8%AF%D9%88%D8%A7%D9%84%D9%8A%D8%A8-%D9%88%D8%B9%D8%B1%D8%A8%D8%A7%D8%AA-%D8%A7%D9%84%D9%82%D9%87%D9%88%D8%A9/c1671312427'},
    {'cat1': 'ركن القهوة', 'cat2': 'اكواب وحامل الاكواب', 'cat3': '', 'url': 'https://spring01.com/%D8%A7%D9%83%D9%88%D8%A7%D8%A8-%D9%88%D8%AD%D8%A7%D9%85%D9%84-%D8%A7%D9%84%D8%A7%D9%83%D9%88%D8%A7%D8%A8/c1031492404'},
    {'cat1': 'ركن القهوة', 'cat2': 'ديكورات القهوة', 'cat3': '', 'url': 'https://spring01.com/%D8%AF%D9%8A%D9%83%D9%88%D8%B1%D8%A7%D8%AA-%D8%A7%D9%84%D9%82%D9%87%D9%88%D8%A9/c1496224054'},
    
    {'cat1': 'فوانيس وشموع ومباخر', 'cat2': 'فوانيس', 'cat3': '', 'url': 'https://spring01.com/%D9%81%D9%88%D8%A7%D9%86%D9%8A%D8%B3/c506034749'},
    {'cat1': 'فوانيس وشموع ومباخر', 'cat2': 'شموع', 'cat3': '', 'url': 'https://spring01.com/%D8%B4%D9%85%D9%88%D8%B9/c1811388734'},
    {'cat1': 'فوانيس وشموع ومباخر', 'cat2': 'مباخر', 'cat3': '', 'url': 'https://spring01.com/%D9%85%D8%A8%D8%A7%D8%AE%D8%B1/c1685281588'},
    
    {'cat1': 'أنتيكات حديد و مجمسات سيراميك', 'cat2': '', 'cat3': '', 'url': 'https://spring01.com/%D8%A3%D9%86%D8%AA%D9%8A%D9%83%D8%A7%D8%AA-%D8%AD%D8%AF%D9%8A%D8%AF-%D9%88-%D9%85%D8%AC%D9%85%D8%B3%D8%A7%D8%AA-%D8%B3%D9%8A%D8%B1%D8%A7%D9%85%D9%8A%D9%83/c987489530'},
    {'cat1': 'منتجات متنوعة', 'cat2': '', 'cat3': '', 'url': 'https://spring01.com/%D9%85%D9%86%D8%AA%D8%AC%D8%A7%D8%AA-%D9%85%D8%AA%D9%86%D9%88%D8%B9%D8%A9/c1172092991'},

]

In [9]:
url = 'https://spring01.com/%D8%B7%D8%A7%D9%88%D9%84%D8%A7%D8%AA-%D9%88%D9%83%D8%B1%D8%A7%D8%B3%D9%8A/c1556539599'

In [10]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
cookies = {'session': '134-8225175-0355220'}
r = requests.get(url, headers=headers, cookies=cookies)
soup = BeautifulSoup(r.content, "html.parser")

In [23]:
def extract_data(url):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    cookies = {'session': '134-8225175-0355220'}
    r = requests.get(url, headers=headers, cookies=cookies)
    soup = BeautifulSoup(r.content, "html.parser")
    name = soup.find('h1', {'class': 'title title--xx-large mb-10'}).text.replace('\u200e', '').strip()
    price = soup.find('div', {'class': 'price-wrapper-info price-wrapper--large'}).text.replace('ر.س', '').strip()
    sku = soup.find('div', {'class': 'list--table-view__cell value text-unicode'}).text.strip()
    description = soup.find('article', {'class': 'article article--main article--product-details mb-50'}).text.strip()
    images = soup.find('div', {'id': 'product_main_slider'}).find_all('img')
    list_images = [img['data-splide-lazy'] for img in images]
    base_image = list_images[0]
    add_images = ','.join(list_images[1: ])
    cats = soup.find('section', {'class': 'breadcrumb'}).find_all('a')

    cat1 = cats[1].text.strip()
    try:
        cat2 = cats[2].text.strip()
    except:
        cat2 = ''
    
    data = {
        'sku': sku,
        'name': name,
        'price': price,
        'link_url': url,
        'description': description,
        'base_image': base_image,
        'add_images': add_images,
        'cat1': cat1,
        'cat2': cat2,
    }
    


'المقاسات ...مقاس اللعبة..20*20..سنتم'

In [12]:
products = soup.find_all('div', {'class': 'product-block__thumb'})
for pro in products:
    print(pro.find('a')['href'])

https://spring01.com/طاولة-خشب-شكل-دراجة-حديد/p1609630051
https://spring01.com/طاولة-حديد-سطح-خشب/p2064975335
https://spring01.com/طقم-طاولات-حديد-سطح-خشب-رخامي-2-قطعة/p579978050
https://spring01.com/طقم-طاولات-حديد-سطح-خشب-رخامي-2-قطعة/p433173776
https://spring01.com/طقم-طاولات-حديد-سطح-خشب-2-قطعة/p993302565
https://spring01.com/طقم-طاولات-حديد-سطح-خشب-رخامي-2-قطعة/p99717779
https://spring01.com/طقم-طاولات-حديد-سطح-خشب-رخامي-2-قطعة/p1233042107
https://spring01.com/طاولة-خدمة-حديد-وخشب/p113381970
https://spring01.com/طقم-طاولات-حديد-سطح-خشب-2-قطعة/p400472677
https://spring01.com/طقم-طاولة-مدخل-حديد-ذهبي-سطح-مراية-عدد-قطعتين/p297748208
https://spring01.com/طقم-طاولة-مدخل-حديد-ذهبي-سطح-مراية-عدد-قطعتين/p532689038
https://spring01.com/طقم-طاولة-مدخل-حديد-ذهبي-سطح-مراية-عدد-قطعتين/p574717871
https://spring01.com/طقم-طاولة-مدخل-حديد-ذهبي-سطح-مراية-عدد-قطعتين/p2144825965
https://spring01.com/طقم-طاولات-حديد-ابيض-ثلاث-قطع/p1763644929
https://spring01.com/طقم-طاولات-حديد-اسواد-ثلاث-قطع/p109138